__Run basic contamination screening and fastqc. perform mapping and alignment, following this tutorial https://www.biostars.org/p/41951/__

In [ ]:
%cd ~/ClownfishGWAS/data/APPC_sequencing/ultraplex_out/fastq_screen/2022-02-07-fastq_screen/


In [ ]:
#make a list of read filenames to use for next loops
!ls *R2.fastq.gz > filenames
!sed -i'' -e 's/R2.fastq.gz//g' filenames #remove the extension

In [ ]:
#run fastqc 
!for i in /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_out/fastq_screen/2022-02-07-fastq_screen/*.fastq.gz; do fastqc -t 34 *.fastq.gz $i; done
#visualize sequencing quality for all samples in trimmed_reads directory. check that all barcode sequences are removed
!multiqc .

In [38]:
#there are some samples fastqc failed for both reads per base seuqence content, use samtools/bamstats to look at how those aligned versus samples that didn't fail
%cd ~/ClownfishGWAS/data/APPC_sequencing/DNA
!samtools stats Pop01_APPC1921.F.bam > troubleshoot_reads/Pop01_APPC1921.F.stats
!samtools stats Pop07_APPC2603.F.bam > troubleshoot_reads/Pop07_APPC2603.F.stats


/local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA


In [39]:
#plot stats
!plot-bamstats -p troubleshoot_reads/Pop01_APPC1921.F_plot troubleshoot_reads/Pop01_APPC1921.F.stats
#!java -Xmx4g -jar BAMStats-1.25.jar -i Pop01_APPC1921.F.bam # double read fail
#!java -Xmx4g -jar BAMStats-1.25.jar -i Pop07_APPC2603.F.bam # only read 2 fail



set terminal png size 600,400 truecolor
             ^
"troubleshoot_reads/Pop01_APPC1921.F_plot-acgt-cycles.gp", line 2: unknown or ambiguous terminal type; type just 'set terminal' for a list

The command exited with non-zero status 256:
	gnuplot troubleshoot_reads/Pop01_APPC1921.F_plot-acgt-cycles.gp

 at /local/home/katrinac/miniconda3/envs/commandline/bin/plot-bamstats line 51.
	main::error("The command exited with non-zero status 256:\x{a}\x{9}gnuplot trouble"...) called at /local/home/katrinac/miniconda3/envs/commandline/bin/plot-bamstats line 317
	main::plot("troubleshoot_reads/Pop01_APPC1921.F_plot-acgt-cycles.gp") called at /local/home/katrinac/miniconda3/envs/commandline/bin/plot-bamstats line 855
	main::plot_acgt_cycles(HASH(0x5621161af1a0)) called at /local/home/katrinac/miniconda3/envs/commandline/bin/plot-bamstats line 34


In [ ]:
 java -Xmx4g -jar ~/miniconda3/envs/bamstats/BamStats-assembly-1.0.1.jar -R ~/ClownfishGWAS/data/APPC_sequencing/Genome/APercula-Reference/APercula-Reference.fai  -o troubleshoot_reads/ -b Pop01_APPC1921.F.bam --bedfile mapped.bed


In [ ]:
 java -Xmx4g -jar ~/miniconda3/envs/bamstats/BamStats-assembly-1.0.1.jar -o troubleshoot_reads/ -i Pop01_APPC1921.F.bam 


In [ ]:
#for some reason, barcodes still in at the start of read2, so use cutadapt to remove the first 6 + reads from the read2 files 5', and 1 bp 3'. read 1 files remove 1 bp 5' and 1 bp 3'. run in the python2.7 environment
!rename '_R' '.R' *.fastq.gz 
for i in $(cat filenames); do cutadapt -u 1 -u -1 -U 7 -U -1 -o $i\.R1.cut.fastq.gz -p $i\.R2.cut.fastq.gz $i\.R1.fastq.gz $i\.R2.fastq.gz; done

In [ ]:
cutadapt -a ADAPTER_FWD -A ADAPTER_REV -o out.1.fastq -p out.2.fastq reads.1.fastq reads.2.fastq

In [ ]:
#screen for contamination and filter out tagged reads simultaneously, filter 1 means keep only reads that mapped exclusively to the reference for A. percula. since this is already done for the read 1 files, just need to do it for the read 2 files that I cut the barcode from  
!for i in /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_out/fastq_screen/*.cut.fastq.gz; do fastq_screen --tag --filter 1 --threads 32 --outdir /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_out/fastq_screen/ $i; done

In [ ]:
#run fastqc 
!for i in /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_out/fastq_screen/*tagged_filter.fastq.gz; do fastqc -t 6 *tagged_filter.fastq.gz  $i; done

In [ ]:
#visualize sequencing quality for all samples in trimmed_reads directory. check that all barcode sequences are removed
!multiqc .

In [ ]:
#so the read 1 and read 2 files still fail qc because the A/T barcode overhang hookup, trim 1 bp on either side of fragments for all read files, check again and then map
!ls *.tagged_filter.fastq.gz  > filenames
!sed -i'' -e 's/.fastq.gz//g' filenames #remove the extension
for i in $(cat filenames); do cutadapt -u 1 -u -1 -o $i.cut.fastq.gz $i.fastq.gz; done
#delete the unfilered fastas from this directory, since they are still in ultraplex out
#remove cut from filenames so names are consistent
!rename '.cut' '' *.fastq.gz 
!rename '.cut' '' *.fastq.gz #second time because the read 2 files have two cut name spaces because they were cut twice
#run fastqc and multiqc again to check we are ready to map
!for i in /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_out/fastq_screen/*tagged_filter.fastq.gz; do fastqc -t 30 *tagged_filter.fastq.gz  $i; done
#make sure to push the multiqc report to github

In [ ]:
!zcat CAP1_APPC_16.11.R1.fastq.gz | head
!zcat CAP1_APPC_16.11.R2.fastq.gz | head



In [40]:
#with the previous pipeline
!zcat /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/Pop01_APPC1611.R1.fq.gz | head
!zcat /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/Pop01_APPC1611.R2.fq.gz | head


@2_2101_24885_8891/1
GGTAGTTCATGCGTAGCCATACCATTAGCCTATTGTTGTGTTGTTAGTGCCTTT
+
FFFFFFFFFFFFFFFFFFFFFFFFFFFFF:FFFF:,FFFFFFFFFFFFFFFFFF
@2_2101_32353_8891/1
CTTTGGTCATTTTCACCTCAACTGGAATCCTTTGGTTTCTGTCCACAATCTTCT
+
FFFFFFFFFFFFFFFFF:FFFFFFFFFFFFFF:FFFFFFFFFFFFFFFFFFFFF
@2_2101_3721_8907/1
GGTTAACAGATGCCTTAAAACTTGTGACCATTAGTCTTGATTTCACATTTAGAC

gzip: stdout: Broken pipe
@2_2101_24885_8891/2
TCACGTACTAACAACACAGCTAACTATGAAGCTAATGACCACTAAAGACACAGTTATTGC
+
FFFFF,FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF
@2_2101_32353_8891/2
TCACGAGGTCTCCCCAAAATCCTGACTTAAACCCATCAAGAACCTGTGGACTGATGAAGA
+
FFFFF,FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF
@2_2101_3721_8907/2
TCACGATTAGACTTTAGAAGATCAAATTAAACAGAAGAGACAATAAAACGATCAAAAAGA

gzip: stdout: Broken pipe


In [48]:
#previous pipeline, earlier step
!zcat /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/trimmed_reads/fastq_screen/Pop01_APPC1611.R1.tagged.fastq.gz | head
!zcat /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/trimmed_reads/fastq_screen/Pop01_APPC1611.R2.tagged.fastq.gz  | head

@2_2101_12816_1016/1#FQST:Amphiprionpercula:1
TGATCTCTGCTCTGCTGGTGTCAGGTTTCACTTTGTTCTAACAGCTCCTTATAT
+
FF:FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF:FFFFFFFFFFFFF
@2_2101_7365_1031/1#FQST:1
CTCTGTTGTTTTGGCTCTAATAACATGAATATTAAACTGAGCGGAGTGACGAAG
+
FFFF,::FFF,FFFFFFFFFFFF,FF,FFFFFFFFF:FFFFFFFFF,FFFFFFF
@2_2101_28989_1031/1#FQST:2
CAACCAGTCAAAGGTTATTACTGAGGATTTTAAACAAAAACATTCAGGCAAACA

gzip: stdout: Broken pipe
@2_2101_12816_1016/2#FQST:Amphiprionpercula:1
TCACGAATCTGAGGTTCTTCAAATAAATTCAGGTTCCTGTCAAAAGGAACCAGGAAATAT
+
FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF:FFFFF:FFFFFFFF
@2_2101_7365_1031/2#FQST:1
TCACGATAAACATATTTTATCTCTAATTAACCTACAACTAGCACCAAAATTACACTTTTT
+
FFF:F:FFFFFFFFFFFF,FFFFFFFFFFFFFFFFFF:FFF:F,:FFFF:FFFFFFF,FF
@2_2101_28989_1031/2#FQST:2
TCACGTGATTACAAAAAAAATCTGTATCGACCCGGAAAAAAAACAAACATGTCAGGCACC

gzip: stdout: Broken pipe


In [42]:
#previous pipeline, later step
!zcat /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/trimmed_reads/Pop01_APPC1611.R1.fq.gz | head
!zcat /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/trimmed_reads/Pop01_APPC1611.R2.fq.gz | head

@2_2101_12816_1016/1
TGATCTCTGCTCTGCTGGTGTCAGGTTTCACTTTGTTCTAACAGCTCCTTATAT
+
FF:FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF:FFFFFFFFFFFFF
@2_2101_7365_1031/1
CTCTGTTGTTTTGGCTCTAATAACATGAATATTAAACTGAGCGGAGTGACGAAG
+
FFFF,::FFF,FFFFFFFFFFFF,FF,FFFFFFFFF:FFFFFFFFF,FFFFFFF
@2_2101_28989_1031/1
CAACCAGTCAAAGGTTATTACTGAGGATTTTAAACAAAAACATTCAGGCAAACA

gzip: stdout: Broken pipe
@2_2101_12816_1016/2
TCACGAATCTGAGGTTCTTCAAATAAATTCAGGTTCCTGTCAAAAGGAACCAGGAAATAT
+
FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF:FFFFF:FFFFFFFF
@2_2101_7365_1031/2
TCACGATAAACATATTTTATCTCTAATTAACCTACAACTAGCACCAAAATTACACTTTTT
+
FFF:F:FFFFFFFFFFFF,FFFFFFFFFFFFFFFFFF:FFF:F,:FFFF:FFFFFFF,FF
@2_2101_28989_1031/2
TCACGTGATTACAAAAAAAATCTGTATCGACCCGGAAAAAAAACAAACATGTCAGGCACC

gzip: stdout: Broken pipe


In [45]:
#okay, from the fastqc html, looks like the read 2 file needed trimming to about 15bp
#get mapping stats for sample
%cd ~/ClownfishGWAS/data/APPC_sequencing/DNA/

!samtools stats  ~/ClownfishGWAS/data/APPC_sequencing/DNA/Pop01_APPC1611.F.bam > troubleshoot_reads/Pop01_APPC1611.F.stats

#trim with cutadapt, and then remap to the reference
#get the mapping stats again

/local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA
[W::sam_read1] Parse error at line 368
Failure while decoding file


In [ ]:
#trim with cutadapt to remove 17 bp 5', and then remap to the reference
cutadapt -U 17 -o  /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/troubleshoot_reads/Pop01_APPC1611.R1.cut.fastq.gz -p /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/troubleshoot_reads/Pop01_APPC1611.R2.cut.fastq.gz  /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/trimmed_reads/Pop01_APPC1611.R1.fq.gz  /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/trimmed_reads/Pop01_APPC1611.R2.fq.gz

In [50]:
#map again
!bwa mem -t 34 -M -R "@RG\tID:$i\tSM:$i\tPL:Illumina" /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/reference.fasta /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/troubleshoot_reads/Pop01_APPC1611.R1.cut.fastq.gz /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/troubleshoot_reads/Pop01_APPC1611.R2.cut.fastq.gz  2> bwa.$i.log | samtools sort -@ 34 -o /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/troubleshoot_reads/Pop01_APPC1611.cut-RG.bam 2> /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/troubleshoot_reads/Pop01_APPC1611.cut.bam.log  

In [51]:
#get mapping stats again
!samtools stats  /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/troubleshoot_reads/Pop01_APPC1611.cut-RG.bam > troubleshoot_reads/Pop01_APPC1611.cut-RG.stats


In [53]:
!bwa mem -t 34 -M -R "@RG\tID:$i\tSM:$i\tPL:Illumina" /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/reference.fasta /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/trimmed_reads/Pop01_APPC1611.R1.fq.gz /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/trimmed_reads/Pop01_APPC1611.R2.fq.gz  2> bwa.$i.log | samtools sort -@ 34 -o /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/troubleshoot_reads/Pop01_APPC1611-RG.bam 2> /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/troubleshoot_reads/Pop01_APPC1611.bam.log  

In [54]:
!samtools stats  /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/troubleshoot_reads/Pop01_APPC1611-RG.bam > troubleshoot_reads/Pop01_APPC1611-RG.stats


In [ ]:
#awesome! looks like both cut and uncut map pretty well. Trim the read 2 files that failed super hard with cutadapt, and then remap
for i in $(cat filenames); do cutadapt -U 7 -o /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/trimmed_reads/mapping/$i\.R1.cut.fastq.gz -p /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/trimmed_reads/mapping/$i\.R2.cut.fastq.gz /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/trimmed_reads/$i\.R1.fq.gz /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/trimmed_reads/$i\.R2.fq.gz; done

In [ ]:
#stopping here 02/08/2022- also the missing file in the DNA directory is the replicate 3601-
Pop16_sample_TAGCTT.1.fq.gz

In [64]:
!pwd


/local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/trimmed_reads/mapping


In [57]:
#redo mapping for all samples
!for i in $(cat filenames); do bwa mem -t 34 -M -R "@RG\tID:$i\tSM:$i\tPL:Illumina" /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/reference.fasta /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/trimmed_reads/mapping/$i.R1.cut.fastq.gz /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/trimmed_reads/mapping/$i.R2.cut.fastq.gz  2> bwa.$i.log | samtools sort -@ 34 -o /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/trimmed_reads/mapping/$i-RG.bam 2> /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/trimmed_reads/mapping/$i.bam.log; done  


In [58]:
#how did it go?
!samtools stats  /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/trimmed_reads/mapping/Pop16_APPC4215-RG.bam > trimmed_reads/mapping/Pop16_APPC4215-RG.stats
!samtools stats  /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/Pop01_APPC2907.F.bam > troubleshoot_reads/Pop01_APPC2907.F.stats
#woohoo! almost 3.5x more mapped reads and 95% of reads properly paired
113175058/33542679

In [59]:
%cd ~/ClownfishGWAS/data/APPC_sequencing/DNA/trimmed_reads/mapping/


/local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/trimmed_reads/mapping


In [77]:
#mark duplicates, from Jon's script, remember to cite Jon and GNU parallel
!cat /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/filenames | parallel -j 8 "java -Xms4g -jar /local/home/katrinac/miniconda3/envs/commandline/bin/picard.jar MarkDuplicates I={}-RG.bam O={}-RGrd.bam M={}_dup_metrics.txt REMOVE_DUPLICATES=true READ_NAME_REGEX=true  OPTICAL_DUPLICATE_PIXEL_DISTANCE=2500 TAGGING_POLICY=OpticalOnly &> md.{}.log"

Academic tradition requires you to cite works you base your article on.
When using programs that use GNU Parallel to process data for publication
please cite:

  O. Tange (2011): GNU Parallel - The Command-Line Power Tool,
  ;login: The USENIX Magazine, February 2011:42-47.

This helps funding further development; AND IT WON'T COST YOU A CENT.
If you pay 10000 EUR you should feel free to use GNU Parallel without citing.

To silence the citation notice: run 'parallel --citation'.



In [66]:
#filter the bam file to remove the duplicates, from Jon's script, remember to cite Jon
#!filter_bam(){ samtools view -@32 -h -q 10 -F 0x100 -F 0x400 $1-RGmd.bam | mawk '$6 !~/[8-9].[SH]/ && $6 !~ /[1-9][0-9].[SH]/'| samtools view -@ 32 -b }
#!export -f filter_bam
!for i in $(cat /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/filenames); do samtools view -@32 -h -q 10 -F 0x100 -F 0x400 $i-RGmd.bam | mawk '$6 !~/[8-9].[SH]/ && $6 !~ /[1-9][0-9].[SH]/'| samtools view -@ 32 -b  > $i\.F.bam; done        

^C


In [ ]:
#check alignments once more
!samtools stats  /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/trimmed_reads/mapping/Pop16_APPC4215.F.bam > trimmed_reads/mapping/Pop16_APPC4215.F.stats

#call genotypes and calculate GL
#use dDocent to call genotypes/snps

In [ ]:
slc25a5

In [69]:
!pwd

/local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/trimmed_reads/mapping


In [75]:
#hmm, filtered bam has nothing. try just removing with picard instead of using Jon's filtering script?
!java -Xms4g -jar /local/home/katrinac/miniconda3/envs/commandline/bin/picard.jar MarkDuplicates I=Pop16_APPC4215-RG.bam O=Pop16_APPC4215-RGmd.bam M=Pop16_APPC4215_dup_metrics.txt REMOVE_DUPLICATES=true READ_NAME_REGEX=true OPTICAL_DUPLICATE_PIXEL_DISTANCE=2500 TAGGING_POLICY=OpticalOnly &> md.Pop16_APPC4215.log   

In [76]:
!samtools stats  /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/trimmed_reads/mapping/Pop16_APPC4215-RGmd.bam > Pop16_APPC4215.RGmd.stats
#yeah, just removing with picard the files look good. 
81998837/113175058 #about 72% of reads retained

In [ ]:
#(commandline) [katrinac@annotate DNA]$ samtools stats filter.merged.bam > samtools_stats
#looks like only 22% reads properly paired- go back to trim the barcodes with cutadapt but definitely in paired end mode!!

In [ ]:
#go back to previous piepline files with correct read names, use cutadapt on these
#something like 
for i in $(cat cutadapt_totrim7); do cutadapt -u 1 -u -1 -U 7 -U -1 -o $i\.R1.cut.fastq.gz -p $i\.R2.cut.fastq.gz $i\.R1.fastq.gz $i\.R2.fastq.gz; done

In [ ]:
#okay, looks like still weird artifacts of library prep, not barcodes, that are present in 68 read files up to position 7 bp on 5' end. move all filtered files to barcodes_cut directory and use cutadapt once more
for i in $(cat reads_to_cut7); do cutadapt -u 7 -o $i.cut.fastq.gz $i.fastq.gz; done
#rename cut files, remove uncut files
#fastqc one more time, multiqc to check, then map

In [ ]:
#for some reason CAP10_APPC_34.13_R1.tagged_filter.fastq.gz was not including in multiqc report, use fastqc to check if it's good to map with
%cd ~/ClownfishGWAS/data/APPC_sequencing/ultraplex_out/fastq_screen/barcodes_cut/

!fastqc -t 30 CAP10_APPC_34.13_R1.tagged_filter.fastq.gz
#yes looks fine, can move on to mapping

In [ ]:
#collate the names of samples that were in the final cutting round of first 7 bp, and the files that were fine and move into a mapping directory
#!mkdir mapping
!for i in $(cat clean_reads); do mv $i mapping/; done
%cd ~/ClownfishGWAS/data/APPC_sequencing/ultraplex_out/fastq_screen/barcodes_cut/mapping


In [ ]:
#make bwa index for reference fasta to use in mapping
!bwa index /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/Genome/reference.fasta


In [ ]:
#remove the underscore after the sample names of files to map because it was a problem in cutadapt loop
!rename '.cut' '' *.fastq.gz 
!ls  *R1.tagged_filter.fastq.gz > filenames
!sed -i'' -e 's/_R1.tagged_filter.fastq.gz//g' filenames #remove the extension

In [ ]:
#map using default parameters but add -M for picard compatibility, make sure to use the tagged/filtered
!for i in $(cat /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_out/fastq_screen/barcodes_cut/mapping/filenames); do bwa mem -t 34 -M -R "@RG\tID:$i\tSM:$i\tPL:Illumina" /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/Genome/reference.fasta $i\_R1.tagged_filter.fastq.gz $i\_R2.tagged_filter.fastq.gz  2> bwa.$i.log | samtools sort -@ 34 -o $i-RG.bam 2>$i.bam.log; done  

In [ ]:
#read1/2 files are no longer ordered the same and so bwa has issues. switch to bowtie which doesn't care about read ordering
#!mkdir /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/Genome/bowtie
%cd ~/ClownfishGWAS/data/APPC_sequencing/ultraplex_out/fastq_screen/barcodes_cut/mapping
!bowtie2-build /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/Genome/APercula-Reference/GCA_003047355.2_Nemo_v1.1_genomic.fna APPC_ref

In [ ]:
#sort the input read files with bbtools
!for i in $(cat filenames); do repair.sh in1=$i\.R1.fastq.gz in2=$i\.R2.fastq.gz out1=repair/$i.fixed.R1.fq out2=repair/$i.fixed.R2.fq outs=repair/$i.singletons.fq repair; done

In [ ]:
!for i in $(cat filenames); do bowtie2 -x APPC_ref -1 $i\_R1.tagged_filter.fastq.gz -2 $i\_R2.tagged_filter.fastq.gz -S $i.sam; done

In [ ]:
#omg stupid I didn't run cutadapt in paired end mode so it messed up the read names. start in ultraplex out directory and figure out what trimming those files needs
%cd ~/ClownfishGWAS/data/APPC_sequencing/ultraplex_out/

!for i in /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_out/fastq_screen/*tagged_filter.fastq.gz; do fastqc -t 6 *tagged_filter.fastq.gz  $i; done

In [ ]:
#make sure to check before running this, can't remember where .sai files are made...
#join pairs, convert from sam to bam files in screen -S samsort
#screen -S samsort
for i in $(cat APPC_sequencing/ultraplex_out/fastq_screen/filenames);
do bwa sampe /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_out/fastq_screen/mapping/reference.fasta $i\.read1.sai $i\.read2.sai $i\.R1.tagged_filter.fastq.gz $i\.R2.tagged_filter.fastq.gz | samtools view -bS - >  ~/ClownfishGWAS/data/APPC_sequencing/DNA/trimmed_reads/mapping/$i\.bam; done



In [ ]:
#remove duplicate reads with picard
java -Xms4g -jar /local/home/katrinac/miniconda3/envs/commandline/bin/picard.jar MarkDuplicates I={}-R2.tagged_filter.fastq.gz O={}-RGmd.bam M={}_dup_metrics.txt OPTICAL_DUPLICATE_PIXEL_DISTANCE=2500 TAGGING_POLICY=OpticalOnly &> md.{}.log   

In [ ]:
#January 24, 2022 using fastp to trim adapters because trimmomatic left the barcode in read 2 file
for i in $(cat /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/filenames); 
    do fastp -i /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/$i.R1.fq.gz -I /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/$i.R2.fq.gz -o /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/fastp/$i.R1.fastp.fq.gz -O /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/fastp/$i.R2.fastp.fq.gz; 
    done



In [ ]:
#January 25, there's still barcodes in the beginning of the read 2 files after fastp adapter trimming. I'm going to try ultraplex instead.
cp ~/ClownfishGWAS/data/APPC_sequencing/novaseq_2021_03_26_SEQ1/bcsplit/*.gz ~/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/
cp ~/ClownfishGWAS/data/APPC_sequencing/novaseq_2021_04_16_SEQ2/bcsplit/*.gz ~/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/

for i in $(cat /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/cap_filenames); 
    do ultraplex -i /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/$i-read-1.fastq.gz -i2 /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/$i-read-3.fastq.gz -b /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/multiplexing/APPCBarcodes.csv -t 8 -d /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/ultraplex_out; 
    done


In [ ]:
#this worked! do this for all capture pools, rename, move this line to the demultiplex notebook, and then redo the fastq, mapping, picard, then angsd and genotype calling
ultraplex -i /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/CAP1-read-1.fastq.gz -i2 /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/CAP1-read-3.fastq.gz -b /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/multiplexing/Cap1_barcodes.csv -t 8 -d /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/ultraplex_out 